<a href="https://colab.research.google.com/github/nem-42098/SL_Proj_Unlearning/blob/main/Unlearn_test_FM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !git clone https://github.com/nem-42098/SL_Proj_Unlearning.git
# import os
# os.chdir('/content/SL_Proj_Unlearning')

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision import models
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from copy import deepcopy
from torch.nn import Module
from torch.nn import CrossEntropyLoss,Softmax
from torch.utils.data import DataLoader,Dataset
from itertools import chain
import numpy as np
from tqdm import tqdm as tq

### Load Pre-Trained VGG network
> #### https://github.com/chenyaofo
> ### Note: There is some issue with using Batch Norm before ReLu as it creates a bias in the network. So people exchange the order between the two for tackling the bias

In [ ]:
### First time when you wan to download the model
device=torch.device('cuda')
model = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_vgg11_bn", pretrained=True)
model
# model=model.to(device)
### For future uses:Loading from the local

# model_1=torch.hub.load("C:/Users/nmura/.cache/torch/hub/chenyaofo_pytorch-cifar-models_master",'hubconf.py',source='local')



Using cache found in /root/.cache/torch/hub/chenyaofo_pytorch-cifar-models_master


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace=True)
    (14): MaxPool2d(ke

### Check which pre-trained model are available:

In [ ]:
torch.hub.list("chenyaofo/pytorch-cifar-models", force_reload=True)

Downloading: "https://github.com/chenyaofo/pytorch-cifar-models/zipball/master" to /root/.cache/torch/hub/master.zip


['cifar100_mobilenetv2_x0_5',
 'cifar100_mobilenetv2_x0_75',
 'cifar100_mobilenetv2_x1_0',
 'cifar100_mobilenetv2_x1_4',
 'cifar100_repvgg_a0',
 'cifar100_repvgg_a1',
 'cifar100_repvgg_a2',
 'cifar100_resnet20',
 'cifar100_resnet32',
 'cifar100_resnet44',
 'cifar100_resnet56',
 'cifar100_shufflenetv2_x0_5',
 'cifar100_shufflenetv2_x1_0',
 'cifar100_shufflenetv2_x1_5',
 'cifar100_shufflenetv2_x2_0',
 'cifar100_vgg11_bn',
 'cifar100_vgg13_bn',
 'cifar100_vgg16_bn',
 'cifar100_vgg19_bn',
 'cifar100_vit_b16',
 'cifar100_vit_b32',
 'cifar100_vit_h14',
 'cifar100_vit_l16',
 'cifar100_vit_l32',
 'cifar10_mobilenetv2_x0_5',
 'cifar10_mobilenetv2_x0_75',
 'cifar10_mobilenetv2_x1_0',
 'cifar10_mobilenetv2_x1_4',
 'cifar10_repvgg_a0',
 'cifar10_repvgg_a1',
 'cifar10_repvgg_a2',
 'cifar10_resnet20',
 'cifar10_resnet32',
 'cifar10_resnet44',
 'cifar10_resnet56',
 'cifar10_shufflenetv2_x0_5',
 'cifar10_shufflenetv2_x1_0',
 'cifar10_shufflenetv2_x1_5',
 'cifar10_shufflenetv2_x2_0',
 'cifar10_vgg11_bn

### Downlaoding the Dataset and Creating the Dataloaders

In [ ]:
### Transformation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
       (0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)
    )
])
### Pytorch Datasets
train_dataset = torchvision.datasets.CIFAR10(
    root= './data', train = True,
    download =True, transform = transform)
test_dataset = torchvision.datasets.CIFAR10(
    root= './data', train = False,
    download =True, transform = transform)
### Dataloaders
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
len(train_dataset)

50000

### Create the Syntheic Dataset
> ### There is parameter equivalent to Noise which informs me about the percentage of the samples whose labels would be randomly generated

In [ ]:
# # Define the classes
# classes = ['forget', 'retain']

# # Create a dictionary to store datasets for each class
# class_datasets = {class_name: [] for class_name in classes}

# # Iterate through the CIFAR-10 dataset and split it into class-specific subsets
# for image, label in train_dataset:
#   if label == 1:
#     class_datasets['forget'].append((image, label))

#   else:
#       class_datasets['retain'].append((image, label))

# # You now have class-specific subsets in the class_datasets dict
import random
torch.manual_seed(0)

# Define the percentage of samples to shuffle labels
shuffle_percentage = 0.2  # Change this value as needed

# Calculate the number of samples to shuffle
num_samples_to_shuffle = int(shuffle_percentage * len(train_dataset))

# Create a copy of the original dataset
synthetic_dataset = deepcopy(train_dataset)

# Shuffle the labels of a subset of samples
indices_to_shuffle = torch.randperm(len(train_dataset))[:num_samples_to_shuffle]
for idx in indices_to_shuffle:
    current_label = train_dataset[idx][1]  # Get the original label
    # Choose a new label randomly (excluding the original label)
    new_label = torch.randint(0, 10, (1,)).item()
    print(new_label)
    while new_label == current_label:
        new_label = torch.randint(0, 10, (1,)).item()
    # Update the label in the synthetic dataset
    synthetic_dataset.targets[idx] = new_label



Streaming output truncated to the last 5000 lines.
9
5
8
6
3
7
4
1
9
1
9
8
6
4
5
3
4
4
6
0
8
1
4
1
9
9
0
8
2
0
2
5
2
9
7
4
1
8
2
8
9
1
8
6
2
8
1
6
4
6
9
4
9
8
4
7
1
4
0
3
3
2
6
6
2
7
5
1
4
9
0
2
5
4
9
1
5
7
6
4
6
7
1
9
2
4
9
2
1
8
5
5
8
6
7
6
0
6
9
4
1
9
6
8
0
4
8
2
5
0
8
0
9
4
4
6
1
1
1
8
7
7
8
3
9
8
4
8
5
9
8
3
5
2
5
9
5
9
6
9
0
9
1
3
4
9
3
5
3
8
5
2
3
2
3
2
4
9
4
3
0
7
0
7
1
0
4
5
9
5
9
4
8
4
0
2
8
3
9
7
5
4
4
3
4
3
9
3
0
7
7
3
7
4
9
8
6
1
9
7
3
5
4
1
6
3
9
4
7
6
9
1
2
9
7
8
4
0
9
9
5
3
8
7
0
6
1
9
9
4
5
2
3
1
8
0
5
4
0
3
7
0
8
8
1
9
9
6
3
6
3
9
9
4
7
9
5
2
5
9
4
2
3
7
3
5
0
6
1
8
7
7
9
4
9
8
2
5
2
8
4
2
0
9
0
6
0
8
5
8
7
9
8
2
5
6
8
4
5
7
6
8
6
3
8
1
3
3
9
6
3
7
7
6
8
7
6
5
1
9
8
6
2
7
1
4
3
8
4
4
5
0
2
7
1
0
1
0
7
4
2
4
8
4
9
2
6
1
5
8
4
8
1
0
4
1
8
7
1
7
5
5
9
7
3
2
6
4
8
4
3
9
5
5
6
2
5
2
9
2
3
2
5
6
1
0
6
4
6
3
7
3
2
5
7
2
7
5
4
9
7
6
4
5
9
5
7
7
6
1
3
1
1
9
3
3
8
8
4
2
1
1
6
4
2
4
8
9
1
8
9
2
7
6
4
0
8
4
8
8
4
7
9
1
6
8
4
3
9
6
1
5
1
4
6
9
8
1
1
8
0
1
3
5
8
9
6
3
1
6
3
3
7
5
6

In [ ]:
### Congiguration file taken
config={'max_epochs': 120,

                     'data':
                      [('type_', 'cifar100'),
                                  ('image_size', 32),
                                  ('num_classes', 100),
                                  ('root', 'data/cifar100'),
                                  ('mean', [0.507, 0.4865, 0.4409]),
                                  ('std', [0.2673, 0.2564, 0.2761]),
                                  ('batch_size', 256),
                                  ('num_workers', 4)],
                     'model':[('type_', 'PyTorchHub'),
                                  ('force_reload', False),
                                  ('repo', 'chenyaofo/pytorch-cifar-models'),
                                  ('name', 'cifar100_vgg11_bn'),
                                  ('pretrained', False)],
                     'optimizer':
                      [('type_', 'SGD'),
                                  ('lr', 0.1),
                                  ('momentum', 0.9),
                                  ('dampening', 0),
                                  ('weight_decay', 0.0005),
                                  ('nesterov', True)],
                     'scheduler':
                      [('type_', 'CosineAnnealingLR'),
                                  ('T_max', 200),
                                  ('eta_min', 0)],
                     }

In [ ]:
def init_weights(layer: Module) -> None:
        """
        reset_parameters = getattr(layer, "reset_parameters", None)
        if callable(reset_parameters):
            layer.reset_parameters()
        if isinstance(layer, nn.BatchNorm2d):
            # Reset BatchNorm parameters (gamma and beta)
            nn.init.ones_(layer.weight)  # Adjust 'a' and 'b' as needed
            nn.init.zeros_(layer.bias)
            # Reset BatchNorm running statistics (mean and variance)
            layer.reset_running_stats()
            return

        if hasattr(layer, 'bias') and layer.bias is not None:
            nn.init.zeros_(layer.bias)
        if hasattr(layer, 'weight'):
            nn.init.xavier_normal_(layer.weight, 1)
        """
        if isinstance(layer, nn.Conv2d):
            nn.init.kaiming_normal_(layer.weight, mode='fan_out', nonlinearity='relu')
            if layer.bias is not None:
                nn.init.constant_(layer.bias, 0)
        elif isinstance(layer, nn.BatchNorm2d):
            nn.init.constant_(layer.weight, 1)
            nn.init.constant_(layer.bias, 0)
        elif isinstance(layer, nn.Linear):
            nn.init.normal_(layer.weight, 0, 0.01)

In [ ]:
def reset_weights(model: Module,device='cuda') -> Module:
        dumb_model = deepcopy(model)
        dumb_model = dumb_model.apply(init_weights)
        return dumb_model.to(device)

In [ ]:
def train_VGG(model:Module,train_dataset:Dataset,config:dict,device:str):

    model=reset_weights(model)
    ### intialising the optimiser
    optimizer = torch.optim.Adam(model.parameters(),lr=1e-3,weight_decay=0.0001) #torch.optim.SGD(model.parameters(), lr=config['optimizer'][2][1],weight_decay=config['optimizer'][-2][1],momentum=config['optimizer'][2][1],dampening=config['optimizer'][3][1])
    ### intialiasing the scheuler
    scheduler=torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,T_max=config['scheduler'][1][1],eta_min=config['scheduler'][2][1])
    ### Number of the epochs
    epochs=config['max_epochs']
    ## Train dataloader
    train_dataloader_syn=DataLoader(train_dataset, batch_size=config['data'][-2][1], shuffle=True, num_workers=2)
    ### early stopping counter
    stop_counter = 0
    ## criterion
    criterion=CrossEntropyLoss()

    ### Epoch Log of losses
    epoch_log=[]



    ###Iterations
    for epoch in tq(range(epochs)):
        ### iterate over the forget_dataloader
        ### batch _loss
        loss_epoch = 0

        model.train()

        for i, (inputs, targets) in enumerate(train_dataloader_syn):


            ### GPU push
            inputs, targets = inputs.to(
                device), targets.to(device)

            if next(model.parameters()).is_cuda:
                    pass
                    # print("Model is on CUDA (GPU)")
            else:
                    model=model.to(device)


            ### predictions from the model
            y_pred = model(inputs)

            optimizer.zero_grad()

            ###Calculate the Loss

            loss = criterion(y_pred, targets)

            ## Backpropogate the Loss
            loss.backward()
            ### Update the weights
            optimizer.step()

            #### Train-set loss
            loss_epoch += loss.item()

        loss_epoch /= (i + 1)

        ### Check for early stopping: if the decrease in the loss is less than 1e-3 for straight 5 iterations

        if epoch > 0:
            if abs(epoch_log[-1][0] - loss_epoch) < 1e-3:

                print('increaing the counter',epoch_log[-1][0] - loss_epoch)
                stop_counter += 1

            else:
                stop_counter = 0


        # print(loss_epoch)
        if epoch%5==0:
          acc_1=test(model,train_dataloader_syn,'cuda')
          acc_2=test(model,trainloader,'cuda')
          epoch_log.append([loss_epoch,acc_1,acc_2])
          sns.lineplot(pd.DataFrame(epoch_log))
          plt.show()
        if stop_counter == 5:

            break

    return model,epoch_log


In [ ]:
def test(model, dataloader,device):
        model.eval()
        tp, n = 0,0
        for X,y in dataloader:
            X,y = X.to(device), y.to(device)

            with torch.no_grad():
                y_pred = model(X)

            tp += (y_pred.argmax(axis=1) == y).sum().item()
            n  += y.size(0)

        return tp/n

In [ ]:
# noisy_model,epoch_log=train_VGG(model,synthetic_dataset,config,'cuda')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# torch.save(noisy_model.state_dict(), 'model_noisy.pt')
# from shutil import copyfile
# copyfile('model_noisy.pt', '/content/drive/My Drive/model_noisy.pt')

In [ ]:
noisy_model_state_dict=torch.load('/content/drive/My Drive/model_noisy.pt')

In [ ]:
noisy_model=reset_weights(model)
noisy_model.load_state_dict(noisy_model_state_dict)

<All keys matched successfully>

In [ ]:
train_dataloader_syn=DataLoader(synthetic_dataset, batch_size=config['data'][-2][1], shuffle=True, num_workers=2)

# test(noisy_model,train_dataloader_syn,'cuda')

In [ ]:
# test(noisy_model,trainloader,'cuda')

In [ ]:
# test(noisy_model,testloader,'cuda')

### Create the noisy Dataset

In [ ]:
noise_index=[]
retain_index=[]
for idx,i in enumerate(train_dataset):

  if i[1] != synthetic_dataset[idx][1]:
    noise_index.append(idx)
  else:
    retain_index.append(idx)


In [ ]:
forget_dataset=torch.utils.data.Subset(synthetic_dataset,noise_index)
forget_dataloader=DataLoader(forget_dataset,batch_size=256,shuffle=True,num_workers=2)
correct_noisy_data=torch.utils.data.Subset(train_dataset,noise_index)

correct_noisy_dataloader=DataLoader(correct_noisy_data,batch_size=256,shuffle=True,num_workers=2)

retain_dataset=torch.utils.data.Subset(train_dataset,retain_index)
retain_dataloader=DataLoader(retain_dataset,batch_size=256,shuffle=True,num_workers=2)

### Noisy models learns the noisy data points

In [ ]:
# test(noisy_model,forget_dataloader,'cuda')

In [ ]:
# test(noisy_model,correct_noisy_dataloader,'cuda')

In [ ]:
from SL_Proj_Unlearning.tools.Unlearner_SGN import Unlearner_SGN
r_perf=[]
r_perf_1=[]

alpha=[1,3,6,9,12]
forget_path='./forget_hess_cifar10_vgg11_random_samples.pt'
retain_path='./retain_hess_cifar10_vgg11_random_samples.pt'
for i in alpha:
    # model = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_vgg11_bn", pretrained=True)
    torch.cuda.empty_cache()
    unlearner = Unlearner_SGN(i,noisy_model, lr = 1e-3)
    ### Getting the new model masked model

    new_model_1=unlearner.Fisher_Masking(retain_dataloader,forget_dataloader,forget_hess_path=forget_path,retain_hess_path=retain_path)
    forget_perf=Unlearner_SGN.test(new_model_1,forget_dataloader,'cuda')
    retain_perf=Unlearner_SGN.test(new_model_1,retain_dataloader,'cuda')
    correct_perf=Unlearner_SGN.test(new_model_1,correct_noisy_dataloader,'cuda')
    test_perf=Unlearner_SGN.test(new_model_1,testloader,'cuda')
    r_perf.append([forget_perf,retain_perf,test_perf,correct_perf])
    print(i,forget_perf,retain_perf,test_perf,correct_perf)
    new_model_1,epoch_log=unlearner.fine_tune(new_model_1,retain_dataloader,30)
    forget_perf=Unlearner_SGN.test(new_model_1,forget_dataloader,'cuda')
    retain_perf=Unlearner_SGN.test(new_model_1,retain_dataloader,'cuda')
    correct_perf=Unlearner_SGN.test(new_model_1,correct_noisy_dataloader,'cuda')
    test_perf=Unlearner_SGN.test(new_model_1,testloader,'cuda')

    r_perf_1.append([forget_perf,retain_perf,test_perf,correct_perf,[epoch_log]])
    print(i,r_perf_1[-1])

22087


/content/SL_Proj_Unlearning/tools/Unlearner_SGN.py:267: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3571.)
  a=((state_dict[k].T.cpu())*Count_damp[idx].T).T


1 0.0979 0.099375 0.1 0.1025


In [ ]:
from SL_Proj_Unlearning.tools.Unlearner_SGN import Unlearner_SGN
r_perf=[]
r_perf_1=[]

alpha=[1,5,10,15]
forget_path='./forget_hess_cifar10_vgg11_random_samples.pt'
retain_path='./retain_hess_cifar10_vgg11_random_samples.pt'
for i in alpha:
    # model = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_vgg11_bn", pretrained=True)
    torch.cuda.empty_cache()
    unlearner = Unlearner_SGN(i,noisy_model, lr = 1e-3)
    ### Getting the new model masked model

    new_model_1=unlearner.Fisher_Masking(retain_dataloader,forget_dataloader,forget_hess_path=forget_path,retain_hess_path=retain_path)
    forget_perf=Unlearner_SGN.test(new_model_1,forget_dataloader,'cuda')
    retain_perf=Unlearner_SGN.test(new_model_1,retain_dataloader,'cuda')
    correct_perf=Unlearner_SGN.test(new_model_1,correct_noisy_dataloader,'cuda')
    test_perf=Unlearner_SGN.test(new_model_1,testloader,'cuda')
    r_perf.append([forget_perf,retain_perf,test_perf,correct_perf])
    print(i,forget_perf,retain_perf,test_perf,correct_perf)
    new_model_1,epoch_log=unlearner.fine_tune(new_model_1,retain_dataloader,10)
    forget_perf=Unlearner_SGN.test(new_model_1,forget_dataloader,'cuda')
    retain_perf=Unlearner_SGN.test(new_model_1,retain_dataloader,'cuda')
    correct_perf=Unlearner_SGN.test(new_model_1,correct_noisy_dataloader,'cuda')
    test_perf=Unlearner_SGN.test(new_model_1,testloader,'cuda')

    r_perf_1.append([forget_perf,retain_perf,test_perf,correct_perf,[epoch_log]])
    print(i,r_perf_1[-1])


Model is on CUDA (GPU)
Finished Computing Hessian Diagonal
Finished Computing Hessian Diagonal
22087
1 0.0981 0.0988 0.0997 0.1029


100%|██████████| 10/10 [02:01<00:00, 12.17s/it]


1 [0.4563, 0.934225, 0.668, 0.2711, [[22079115198.375237, 4364.729548504019, 857.4245083024547, 21.976904417583896, 1.9169241517402564, 3.473966768402962, 256.7299280974326, 14.454121624113649, 0.2975138479926784, 300.67758303728834]]]
8862
5 0.0978 0.0996 0.0996 0.1025


100%|██████████| 10/10 [02:04<00:00, 12.48s/it]


5 [0.306, 0.80215, 0.5812, 0.3076, [[76259613738548.61, 683988159.7231646, 11420055.812253425, 941085282.083149, 20561.768335829496, 9.198609800855065, 60.38766669467756, 592.9110322807245, 0.791780934971609, 0.9650529205419456]]]
6750
10 0.0979 0.09935 0.0999 0.1025


100%|██████████| 10/10 [02:03<00:00, 12.35s/it]


10 [0.2247, 0.684375, 0.508, 0.2888, [[7.48839312144674e+17, 31931.4677783919, 10555.933420548774, 12351.202237782205, 1.0646153240446832, 8.45377278176083, 0.94482392604184, 20417170049551.48, 39572.41420651925, 1.2666907523088395]]]
5996
15 0.0979 0.099375 0.1 0.1025


100%|██████████| 10/10 [02:16<00:00, 13.68s/it]


15 [0.0979, 0.099375, 0.1, 0.1025, [[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]]]


In [ ]:
from SL_Proj_Unlearning.tools.Unlearner_SGN import Unlearner_FM

r_perf_1=[]

R=list(np.linspace(0.001,0.01,10))
R+=list(np.linspace(0.01,0.1,10))
R+=[0.2,0.4,0.6]
forget_path='./forget_hess_cifar10_vgg11_random_samples.pt'
retain_path='./retain_hess_cifar10_vgg11_random_samples.pt'
for i in R:
    # model = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_vgg11_bn", pretrained=True)
    torch.cuda.empty_cache()
    unlearner = Unlearner_FM(i,noisy_model, lr = 1e-3)
    ### Getting the new model masked model

    new_model_1=unlearner.Fisher_Masking(retain_dataloader,forget_dataloader,forget_hess_path=forget_path,retain_hess_path=retain_path)

    new_model_1,epoch_log=unlearner.fine_tune(new_model_1,retain_dataloader,30)
    forget_perf=Unlearner_FM.test(new_model_1,forget_dataloader,'cuda')
    retain_perf=Unlearner_FM.test(new_model_1,retain_dataloader,'cuda')
    correct_perf=Unlearner_FM.test(new_model_1,correct_noisy_dataloader,'cuda')
    test_perf=Unlearner_FM.test(new_model_1,testloader,'cuda')

    r_perf_1.append([forget_perf,retain_perf,test_perf,correct_perf,[epoch_log]])
    print(i,forget_perf,retain_perf,test_perf,correct_perf)


In [ ]:
torch.save(r_perf_1, 'model_FM_unlearning_random_samples.pt')
from shutil import copyfile
copyfile('model_FM_unlearning_random_samples.pt', '/content/drive/My Drive/model_FM_unlearning_random_samples.pt')

In [ ]:
plt.figure(figsize=(10,5))
df =pd.DataFrame(r_perf_1)
R_=[ round(i,3) for i in R ]
df.index=[ i for i in R_]
df.columns=['forget_noisy_random_labels','retain_data','noisy_correct_data_labels','test_data','epch_log']
sns.lineplot(data=df.iloc[:,0:4])
plt.xscale('log')
plt.xticks(rotation=90)
plt.xlabel('Remvoal Ratio')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

#### Forget and Retain

In [ ]:
# Class split
retain_dataloader = torch.utils.data.DataLoader(class_datasets['retain'], batch_size=128, shuffle=True, num_workers=2)
forget_dataloader = torch.utils.data.DataLoader(class_datasets['forget'], batch_size=128, shuffle=True, num_workers=2)

# Random split
# train_split_dataset,forget_split_dataset=torch.utils.data.random_split(train_dataset,lengths=[45000,5000])
# retain_dataloader = torch.utils.data.DataLoader(train_split_dataset,  batch_size=128, shuffle=True, num_workers=2)
# forget_dataloader = torch.utils.data.DataLoader(forget_split_dataset, batch_size=128, shuffle=True, num_workers=2)

# Unlearner class


### Using the Lower precision based model.
> ### There is some problems with this method. Because the counterpart gives better result.

In [ ]:
from tools.Unlearner_FM_precision import Unlearner_FM_precision
r_perf=[]

R=list(np.linspace(0.001,0.1,10))
R+=list(np.linspace(0.01,0.1,10))
R+=[0.2,0.4,0.6]
for i in R:
    model = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_vgg11_bn", pretrained=True)
    torch.cuda.empty_cache()
    unlearner = Unlearner_FM_precision(i,model, lr = 1e-6)
    ### Getting the new model masked model

    new_model, mask_index,num_of_param=unlearner.Fisher_Masking(retain_dataloader,forget_dataloader)

    forget_perf=Unlearner_FM_precision.test(new_model,forget_dataloader,'cuda')
    retain_perf=Unlearner_FM_precision.test(new_model,retain_dataloader,'cuda')
    test_perf=Unlearner_FM_precision.test(new_model,testloader,'cuda')

    r_perf.append([forget_perf,retain_perf,test_perf])




In [ ]:
r_perf

In [ ]:
r_perf

### Counterpart: Maintaining the original precision of the model and diverting some operation to CPU.

In [ ]:
# from tools.Unlearner_FM import Unlearner_FM
# torch.cuda.empty_cache()
# model = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_vgg11_bn", pretrained=True)
# unlearner_1 = Unlearner_FM(0.02,model, lr = 1e-6)


from tools.Unlearner_FM import Unlearner_FM
r_perf_1=[]

R=list(np.linspace(0.001,0.01,10))
R+=list(np.linspace(0.01,0.1,10))
R+=[0.2,0.4,0.6]
forget_path='./data/forget_hess_cifar10_vgg11_class1.pt'
retain_path='./data/retain_hess_cifar10_vgg11_class1.pt'
for i in R:
    model = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_vgg11_bn", pretrained=True)
    torch.cuda.empty_cache()
    unlearner = Unlearner_FM(i,model, lr = 1e-6)
    ### Getting the new model masked model

    new_model_1, mask_index_1,num_of_param_1=unlearner.Fisher_Masking(retain_dataloader,forget_dataloader,forget_hess_path=forget_path,retain_hess_path=retain_path)


    forget_perf=Unlearner_FM.test(new_model_1,forget_dataloader,'cuda')
    retain_perf=Unlearner_FM.test(new_model_1,retain_dataloader,'cuda')
    test_perf=Unlearner_FM.test(new_model_1,testloader,'cuda')

    r_perf_1.append([forget_perf,retain_perf,test_perf])



In [ ]:
R_=[ round(i,3) for i in R ]

In [ ]:
plt.figure(figsize=(10,5))
df =pd.DataFrame(r_perf_1)
df.index=[ i for i in R_]
df.columns=['forget','retain','test']
sns.lineplot(data=df)
plt.xscale('log')
plt.xticks(rotation=90)
plt.xlabel('Remvoal Ratio')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
r_perf_1

In [ ]:
from tools.Unlearner_FM import Unlearner_FM
r_perf_2=[]

R=list(np.linspace(0.001,0.01,10))
R+=list(np.linspace(0.01,0.1,10))
R+=[0.2,0.4,0.6]
forget_path='./data/forget_hess_cifar10_vgg11_class1.pt'
retain_path='./data/retain_hess_cifar10_vgg11_class1.pt'
for i in R:
    model = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_vgg11_bn", pretrained=True)
    torch.cuda.empty_cache()
    unlearner = Unlearner_FM(i,model, lr = 1e-6)
    ### Getting the new model masked model

    new_model_1, mask_index_1,num_of_param_1=unlearner.Fisher_Masking(retain_dataloader,forget_dataloader,forget_hess_path=forget_path,retain_hess_path=retain_path)

    new_model_1,epoch_log=unlearner.fine_tune(new_model_1,retain_dataloader)
    forget_perf=Unlearner_FM.test(new_model_1,forget_dataloader,'cuda')
    retain_perf=Unlearner_FM.test(new_model_1,retain_dataloader,'cuda')
    test_perf=Unlearner_FM.test(new_model_1,testloader,'cuda')

    r_perf_2.append([forget_perf,retain_perf,test_perf,[epoch_log]])


## Run the unlearning


In [ ]:
### Getting the new model masked model

new_model_1, mask_index_1,num_of_param_1=unlearner_1.Fisher_Masking(retain_dataloader,forget_dataloader)

In [ ]:
Unlearner_FM.test(new_model_1,forget_dataloader,'cuda')

In [ ]:
Unlearner_FM.test(new_model_1,retain_dataloader,'cuda')

In [ ]:
Unlearner_FM.test(new_model_1,testloader,'cuda')

We can see that the unlearning process may need many epochs in the erasure phase to converge. We could also try to increase the learning rate for faster convergence.

In [ ]:
import pandas as pd
import seaborn as sns

log_df = pd.DataFrame.from_records(unlearner.log, columns=['phase', 'epoch', 'batch', 'tp', 'n', 'loss'])
unlearn_history = log_df.groupby(['phase', 'epoch']).agg({'tp':sum, 'n':sum, 'loss': 'mean'}).reset_index()
unlearn_history['accuracy'] = unlearn_history.tp / unlearn_history.n
unlearn_history.loc[unlearn_history.phase == 'erasure', 'loss_scale'] = unlearn_history.loc[unlearn_history.phase == 'erasure', 'loss'] /unlearn_history.loc[unlearn_history.phase == 'erasure', 'loss'].max()
unlearn_history.loc[unlearn_history.phase == 'retrain', 'loss_scale'] = unlearn_history.loc[unlearn_history.phase == 'retrain', 'loss'] /unlearn_history.loc[unlearn_history.phase == 'retrain', 'loss'].max()

sns.lineplot(data = unlearn_history, x = 'epoch', y='loss_scale', hue='phase')

## Check model performance
We now try check the model performance on the retain and forget set.

In [ ]:
models = {'original':model, 'dumb':unlearner.dumb_model, 'erased':unlearner.erased_model, 'retrained':unlearner.retrained_model}
dataloaders = {'retain':retain_dataloader, 'forget':forget_dataloader, 'test':testloader}

performances = []
for model_name, m in models.items():
  for dl_name, dl in dataloaders.items():
    acc = test(m, dl)
    performances.append((model_name, dl_name, acc))

Strangely the dumb network has 0% performance on the forget set, and this may have a negative impact, note that the performance on the forget set is even too low (we expected 10%, not less).

Outside of that we note that this procedure works great for the retain and test which are fitted very nicely, and we can even see that there's a big leap in performance from the erased model to the retrained model.

In [ ]:
perf_df = pd.DataFrame.from_records(performances, columns=['model', 'data_partition', 'accuracy'])

tb = pd.pivot_table(perf_df, index='data_partition', columns='model', values='accuracy')
sns.heatmap(tb, annot = True, fmt='.2%')